# Paralelni algoritmi - drugi projekat

- Projekat se radi individualno ili u paru

- Rok za predaju projekata je petak 17.12.2021.

- Raspored odbrana će biti objavljen u ponedeljak 20.12.2021.

- Predaja projekata je putem emaila, pri čemu programski kod bi trebalo da
bude prikačen uz sam email. 

- **Rešenje mora da bude obliku jedne python skripte (sa .py ekstenzijom), pri čemu ime fajla treba da bude ime_prezime_bbb.gg.py** (gde je bbb broj indeksa, a gg godina upisa).

- Ukoliko dva studenta rade u paru, dovoljno je da jedan od njih preda, pri čemu šalje dva fajla (za svakoga poseban fajl, nazvan prema uputstvu) i 
drugog stavlja u cc. E-mail treba da sadrzi sve podatke (ime, prezime,
broj indeksa) za oba studenta.

# Zadatak

U okviru projekta implementirate BPA tokenizator, koristeći MapReduce paradigmu. BPA tokenizator prvo analizira uzorak teksta kako bi identifikovao reči ili delove reči koji se često pojavljuju, a potom njih koristi kako bi tesktove pretvorio u vektore brojeva, koji se dalje mogu koristiti za statističke analize i mašinsko učenje.
Cilj projekta je uvežbavanje MapReduce programiranja, pa su neki zahtevi uprošćeni ili prilagođeni MapReduce kodu, dok će neki delovi verovatno biti previše spori za realnu upotrebu. 

## 1. Pristup tekstu sa Wikipedije putem MapReduce programa (3 boda)
Napisati program koji kroz map-reduce paradigmu (upotrebom funkcije `map` iz Python standardne biblioteke i funkcije `reduce` iz paketa `functools`):
  - Za zadatu listu ključnih reči vrši pretragu Wiki stranica i dohvata zadati broj rezultata (naslova stranica)
    Za pristup wiki stranicama se može koristiti python biblioteku [*wikipedia*](https://pypi.org/project/wikipedia/) (pip install wikipedia):
```python
import wikipedia
wikipedia.set_lang("sr")  
_
def get_pages(query, results=50):
  '''Dohvata naslove zahtevanog broja stranica koje se pojavaljuju kao 
      rezultati pretrage za zadatu kljucnu rec'''
      titles = wikipedia.search(query, results=results)
      pages = []
      for title in titles:
          page = wikipedia.page(title)
          pages.append(page)
      return pages
```

- Neki naslovi koje ćemo dobiti pozivom `search` funkcije neće biti validni, pa će poziv `wikipedia.page(title)` baciti izuzetak. Izmeniti rešenje tako da se takvi naslovi odbace, bez propuštanja izuzetka. **0.5 bodova**

- Pri kasnijoj obradi, biće nam zgodnije da umesto liste tesktova ima listu parova (ključ, tekst), gde je ključ ključna reč čijom pretragom smo došli do datog teksta. Modifikovati kod tako da vraća ovavu listu. **0.5 bodova**

- Pomoću funkcije map i funkcije iz prethodne tačke napisati kod koji pretražuje za sledeći skup ključnih reči `['Beograd', 'Prvi svetski rat', 'Protein', 'Mikroprocesor', 'Stefan Nemanja', 'Košarka']`, za svaku dohvata po (maksimalno) 50 rezultata. Rezultat treba da bude lista, koja za svaku traženu ključnu reč sadrži listu ranije difinisanih `tuple`-ova (ključ, tekst). Izvršavanje može da potraje, a kako se uglavnom čeka na odgovor servera, može se ubrzati upotrebom map metoda iz pool-a koji koristi veći broj niti (`multiprocessing.pool.ThreadPool`) sa većim brojem (na primer 32) niti. **1 bod**

- Pomoću funkcije reduce pretvoriti listu listi iz prethodne tačke u listu koja direktno sadrži sve tuplove (tj. spojiti sve "unutrašnje" liste u jednu, operacija koja se nekad naziva i `flatten`). **1 bod**


## 2. Pretprocesiranje teksta (3 boda)

Pre dalje obrade potrebno je pripremiti tekst.
- Napisati kod koji pomoću funkcije `map` zamenjuje sva slova malim slovima. Kao ulaz očekivati tekst u obliku jednog niza karaktera (jedan `string`). Funkcija koja se mapira bi trebalo da radi na nivou jednog karaktera, i može koristiti python metod `str.lower`. Izalaz bi trebalo da bude u obliku liste karaktera. **0.5 bod**

- Napisati kod koji pomoću funkcije `reduce` uklanja interpunkcijske znake i specijalne karaktere. Skup karaktera koji se uklanja možete sami odrediti, a najmanje treba da sadrži: `'=', '-', ',', '!', '?', '.', '$', '(', ')', '[', ']'`. Kao ulaz očekivati tekst u obliku liste karaktera. **1 bod**

- Napisati kod koji pomoću funkcije `reduce` u listu karaktera dodaje jedan specijalni karakter posle svaki reči (graničnik ili "terminator" reči). Najbolje upotrebiti neki od karaktera koji su prethodno izbačeni iz teksta (tako smo sigurni da nije u pitanju karakter iz teksta). U praksi se često koristi karakter "$". Karaktere koji u originalnom tekstu označavaju krajeve reči (npr. " ", "\n", "\t") ovom prilikom izbaciti. Graničnik na samom kraju liste je dozvoljeno dodati posebno, klasičnim kodom. **0.5 bod**

- Kasniji koraci u izvršavanju će biti komputaciono zahtevni. Kako bi ogrnačili vreme izvršavanja, dobro je da ograničimo i dužinu teksta sa kojim radimo. Napisati kod koji pomoću funkcije `reduce` niz iz prethodne tačke skraćuje na otprilike 10.000 karaktera, tako da se uvek završrava graničnikom reči (na primer, kad se stigne do 10.000 karaktera nastavlja se do sledećeg graničnika i tu staje. **0.5 bodova**

- Napisati funkciju koja za dati tekst poziva prethodna 4 koraka pretprocesiranja, i vraća obrađenu listu karaktera. Pomoću funkcije `map`, primeniti ovu funkciju na listu tekstova sa wikipedia-e, iz prvog dela zadataka. Rezultat bi trebalo da bude u vidu liste parova (ključ, niz karaktera), gde je ključ ključna reč čijom pretragom smo došli do datog teksta. **0.5 bodova**


## 3. Kreiranje BPA tokenizatora (6 bodova)

- Tokenizator ćemo kreirati ("trenirati") na manjem podskupu tesktova koje smo pripremili.   
Pomoću funkcije reduce, napisati kod koji iz prethodne stavke nasumično odabira po 5 tesktova iz za svaku pretraženu ključnu reč (prihvatljivo je da to budu i prvih 5 po redosledu kojim su dati u listi, ukoliko usmeno možete da skicirate rešenje koje bi bilo bliže nasumičnom odabiranju). Rezultate bi trebalo da bude *jedna prosta lista karaktera* (odbacujemo ključeve, i sve spajamo u jednu listu). **1 bod**

Sada ćemo napisati kod koji izvršava jednu iteraciju treniranja BPA tokenizatora.

- Pomoću funkcije `reduce`, napisati kod koji za datu listu generiše listu kandata za sledeći token koji će biti dodat u listu tokena, tako što za svaki par x<sub>i</sub>, x<sub>i+1</sub> emituje njihovu konkatenaciju. Na primer, za niz `['A', 'B', 'C', 'A', 'B']` rezultat bi bio `['AB', 'BC', 'CA', 'AB']`. Voditi računa da se graničnik reči može pronaći samo na poslednjem mestu u tekstu tokena (Na primer, za ulaz `['A', '$', 'B', 'C']` rezultat bi trebalo da bude `['A$', 'BC']`). **1 bod**

- Pomoću funkcija `map`, `sorted`, i `reduce` (dozvoljeno je korišćenje python funkcije sorted između map i reduce faza) napisati koji broji pojavljivanja pojednih kandandata za sledeći token, u obliku niza tuplova (token, broj pojavljivanja). Na primer, za niz `['AB', 'BC', 'CA', 'AB']` rezultat bi bio `[('AB', 2), ('BC', 1), ('CA', 1)]`. **2 boda**

- Pomoću funkcije `sorted` sortirati niz iz prethodne tačke po broju pojavljivanja tokena, uzeti najčešći, i dodati ga u trenutnu listu tokena.  
*Klasični BPA tokenizator polazi od liste slova, dok mi možemo krenuti od prazne liste. Takođe klasični BPA će kod dodavanja tokena smanjiti brojače tokena čijim je spajanjem nastao, a one čiji je broj stigao do 0 izbaciti; ni o ovome ne moramo voditi računa, pošto smo dodali korak na kraju koji rešava oba problema po cenu dodatnog računanja*

- Pomoću funkcije `reduce` proći kroz listu karaktera koju koristimo pri za treniranje i ubaciti novi token gde god se pojavljuje. Na primer, ako ubacujemo token 'AB' u listu `['A', 'B', 'C', 'A', 'B']` rezultat bi trebalo da bude `['AB', 'C', 'AB']`. Dobijena lista će biti ulaz u sledeću iteraciju.
U kasnijim iteracijama ubačene tokene tretiramo isto kao i ostale karaktere; na primer za ulaz `['AB', 'C', 'AB']` prvi korak generiše `['ABC', 'CAB']` i tako dalje. **2 boda**

- Klasičnom **for** petljom izvršiti 5000 iteracija treniranja BPE tokenizatora.   
 **Ovaj korak može biti spor, u zavisnosti od toga koliko je optimalno napisan kod, od 15 minuta pa na više.** Dok razvijate kod, možete spustiti broj iteracija na 10-50. Rezultat će biti uglavnom kratki tokeni, dok se posle par hiljada iteracija počinju pojavljivati cele reči (praktični BPE tokenizatori se često treniraju 50.000+ iteracija, ali je i njhov kod optimalnije napisan). 

- Pomoću funkcija `map`, `sorted`, i `reduce`  (dozvoljeno je korišćenje python funkcije sorted između map i reduce faza) napisati kod koji broji pojavljivanja pojednih elemenata u listi dobijenoj transformacijom ulaznog teksta kroz iteracija treniranja.   
Elementi ulazne liste su tokeni i pojednični karakteri koji do sada nisu zamenjeni tokenima. Pojedinačne karaktere tretirati isto kao i tokene (u ovom koraku ćemo ih praktično dodati u listu tokena).  
Rezultat bi trebalo da bude lista u obliku (token, broj pojavljivanja). **1 bod**

- Sortirati listu iz prethodne stavke po dužini tokena. Ova lista nam konačni skup tokena u tokenizatoru (primetite da smo na ovaj način i odbacili sve tokene koji su dodati u procesu treniranja, a više se ne pojavljuju u tekstu, pošto su uključeni u duže tokene).

## 4. Primena BPA tokenizatora (8 bodova)

U ovom delu zadatka se vraćamu na punu listu pretprocesiranih tekstova u obliku liste parova (ključ, lista karaktera) iz prvog dela zadatka.

- Pomoću funkcije `reduce` napisati kod koji prolazi kroz listu karaktera (i potencijalno već ubačenih tokena) i token, i ubacije dati token u izlaznu listu gde god se on pojavljuje. Ova funkcija može zameniti samo sukcesivne karaktere novim tokenom (već ubačeni tokeni ne mogu biti zamenjeni). 
Kod bi trebalo da bude u obliku funkcije koja na ulazu prima listu već obrađenih elemenata ulaza (eventualno u tuple-e sa nekim brojačem ili flag promenljivom), sledeći karakter iz ulazen liste, i token koji se trenutno obradjuje. Upotrebom funckije `partial` iz paketa `functools` ova funkcija bi trebalo da može da se pretvori u funckiju koja je kompatibilna sa reduce. **2 boda**
```python
def insert_token(accumulator, value, token):
...
inserter = partial(insert_token, token=token)
tokenized = reduce(inserter, tokenized, ([], 0))
```
  **Kod implementacije ove funkcije je potreno voditi računa o kompleksnoti, pošto ćemo je izvršavati za svaki tekst i za svaki token, a sama funkcija iterira kroz tekst slovo po slovo.** Jednom kada ste sigurni da radi, možete joj dodati dekorator `@jit` dekorator iz `numba` paketa, kako bi dobili znatno brži, kompajlirani kod. Bez numbe, obrara može potrejati i preko 1h (ili mnogo sati), u zavisnosti od implementacije.

- Pomoću funckije `map` primeniti funkciju iz prethodne tačne na sve ulazne tekstove. Rezultat bi trebalo da bude u vidu liste parova (ključ, niz tokena), gde je ključ ključna reč čijom pretragom smo došli do datog teksta. Izvršavanje može da potraje, pa je dobro ovaj map zameniti map-om koji radi sa više procesa (na Colabu samo 2 procesa, pošto imamo samo dva jezgra). **1 bod**

- Pomoću funkcija `map` i `reduce` izbrojati pojavljivanje **svih** tokena iz liste tokena, u svakom od tekstova. Rezultat treba da bude u vidu liste parova (ključ, lista broja pojavlivanja), pri čemu:
  - Svakom ulaznom tekstu odgovara jedan element liste
  - Ključevi su ključne reči čijom pretragom smo dobili dati tekst
  - Lista broja tokena treba da sadrži sve token (0 za one koji se pojavljuju u datom tekstu), i mora da bude isto sortirana za sve tekstove (ako prvi broj u nizu za prvi tekst označava koliko se puta reč "grad" pojavila u prvom tekstu, onda prvi broj u listama svih ostalih tekstova takođe označava koliko se puta reč "grad" pojavila u datom tekstu). **4 boda**  
  Moguća ideja za rešenje (za jedan tekst) se bazira na map/reduce implementaciji `join`-a:
    - Metodom `map` pretvoriti listu tokena u listu parova ('A', (token, 0)). (ili poći od list tokena koja sadrži broj pojavljivaja tokena u toku treniranja, u kom slučaju broj 0 ne moramo da dodajemo)

    - Pomoću funkcija `map`, `sorted`, i `reduce` napisati kod koji broji pojavljivanja tokena u tokenizovanom tekstu Rezultat treba da bude u vidu liste parova [token, broj pojavljivanja], (tokeni koji se ne pojavljuju u tekstu ne neće pojaviti u rezultatu, redosled tokena je nasumičan). 

    - Metodom `map` pretvoriti listu iz prethodne stavke u listu parova ('B', (token, N)), gde je N broj pojavljivanja datog tokena.

    - Spojiti liste iz prve i treće stavke (list_a + list_b) i sortirati dobijenu listu po vrednosti tokena, pa po labeli ("A" ili "B"). Rezultat je lista koja sadrži:
      - Svaki token iz liste tokena, praćen brojem pojavljivanja u datom tekstu, kao susedne elemente
      - Tokene koji nisu u pronadjeni u tekstu, praćene nečim drugim
      - Karaktere pronadjene u tekstu koji nisu u listi tokena, ispred kojih se neće nalaziti odgovorajući toke
    - Pomoću funkcije reduce proći kroz ovakvu listu i formirati traženi rezultat.

- Pomoće metode `map` primeniti kod koji vrši tokenizaciju na sve tekstove. **1 bod**

## Klasterovanje
(Ne nosi bodove, nije neophodno)

Kod dat u nastavku mozete iskoristiti za evaluaciju dobijenih embedinga tekstova. Kod očekuje list bpr, čiji su elementi parovi (klučna reč, bpr tokenizovan tekst) i vrši klasterovanjem k-means algoritmom u 6 klastera. *Kako je priprema teksta relativno jednostavna, ne očekuju se savršeni rezultati*.

In [ ]:
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import numpy as np
from collections import Counter

labels = [x[0] for x in bpes]
embeddings = np.array([x[1] for x in bpes])

pca_embeddings = PCA(n_components=50).fit_transform(embeddings)
clusters = KMeans(n_clusters=6).fit_predict(pca_embeddings)
Counter(zip(labels, clusters))